In [2]:
# !pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 MB 2.1 MB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 9.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-4.0.0-py2.py3-none-any.whl size=434741238 sha256=70643c862d3d8fd9a9af655ef42fce23ce12967dd0a582e76a681926c66ea416
  Stored in directory: /home/jovyan/.cache/pip/wheels/91/e4/c1/3c917d48563ae77204dd185aa3da90da6a1a5526565296dadf
Successfully built pyspark


In [1]:
import os
from pyspark.sql import SparkSession

MINIO_ACCESS_KEY = os.getenv("MINIO_ROOT_USER", "minio")
MINIO_SECRET_KEY = os.getenv("MINIO_ROOT_PASSWORD", "minio12345678")

spark = (
    SparkSession.builder
    .appName("sensor-dq-notebook")
    .master("local[*]")
    .config(
        "spark.jars.packages",
        ",".join([
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1",
            "org.apache.hadoop:hadoop-aws:3.3.4",
            "com.amazonaws:aws-java-sdk-bundle:1.12.262",
        ])
    )
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY)
    .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY)
    .getOrCreate()
)
print("Spark", spark.version)


:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-05913482-6324-43e9-bf78-fcfc9569d34c;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.ap

Spark 3.5.1


In [2]:
from pyspark.sql import functions as F

curated = spark.read.parquet("s3a://rt-stream/curated")
curated.printSchema()
curated.orderBy(F.col("window_start").desc()) \
       .show(10, truncate=False)


25/08/25 06:54:50 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


root
 |-- device_id: string (nullable = true)
 |-- n_events: long (nullable = false)
 |-- avg_temperature_c: double (nullable = true)
 |-- max_vibration_g: double (nullable = true)
 |-- window_start: timestamp (nullable = true)
 |-- window_end: timestamp (nullable = true)
 |-- date: date (nullable = true)
 |-- hour: integer (nullable = true)



+---------+--------+-----------------+---------------+-------------------+-------------------+----------+----+
|device_id|n_events|avg_temperature_c|max_vibration_g|window_start       |window_end         |date      |hour|
+---------+--------+-----------------+---------------+-------------------+-------------------+----------+----+
|MTR-67420|37      |75.16081081081083|1.068          |2025-08-24 14:20:00|2025-08-24 14:20:30|2025-08-24|14  |
|MTR-60430|42      |74.04738095238096|0.995          |2025-08-24 14:20:00|2025-08-24 14:20:30|2025-08-24|14  |
|MTR-33679|39      |76.03358974358976|1.191          |2025-08-24 14:20:00|2025-08-24 14:20:30|2025-08-24|14  |
|MTR-69765|31      |76.56483870967742|0.269          |2025-08-24 14:20:00|2025-08-24 14:20:30|2025-08-24|14  |
|MTR-36691|47      |76.31978723404255|0.215          |2025-08-24 14:20:00|2025-08-24 14:20:30|2025-08-24|14  |
|MTR-25650|46      |75.71978260869567|1.191          |2025-08-24 14:20:00|2025-08-24 14:20:30|2025-08-24|14  |
|

In [3]:
from pyspark.sql import functions as F

dq = spark.read.json("s3a://rt-stream/dq/streaming_summary")
# Prefer to sort by file mod time if no 'ts' column is present:
if "ts" in dq.columns:
    dq.orderBy(F.col("ts").desc()).show(20, False)
else:
    dq.withColumn("_file", F.input_file_name()) \
      .orderBy(F.col("_file").desc()) \
      .show(20, False)


+--------+----------+-------------+--------+-----------+-----------------+----+------------------------+----------+
|bad_json|bad_status|bad_vibration|epoch_id|missing_key|out_of_range_temp|rows|ts                      |date      |
+--------+----------+-------------+--------+-----------+-----------------+----+------------------------+----------+
|0       |303       |0            |10      |0          |0                |3000|2025-08-24T14:24:21.417Z|2025-08-24|
|0       |282       |0            |9       |0          |0                |2996|2025-08-24T14:21:56.826Z|2025-08-24|
|0       |0         |0            |8       |0          |0                |4   |2025-08-24T14:19:50.242Z|2025-08-24|
|0       |305       |0            |7       |0          |0                |2996|2025-08-24T14:06:59.491Z|2025-08-24|
|0       |1         |0            |6       |0          |0                |4   |2025-08-24T14:05:00.896Z|2025-08-24|
|0       |281       |0            |5       |0          |0               

In [5]:
from pyspark.sql import functions as F

paths = {
    "invalid_temp": "s3a://rt-stream/dq/quarantine/invalid_temp",
    "missing_key":  "s3a://rt-stream/dq/quarantine/missing_key",
}
for name, p in paths.items():
    try:
        n = spark.read.parquet(p).count()
        print(f"{name}: {n}")
    except Exception as e:
        print(f"{name}: (not found yet)")
bad_temp = spark.read.parquet("s3a://rt-stream/dq/quarantine/invalid_temp")
bad_temp.orderBy(F.desc("_metadata.file_modification_time")).show(10, False)


invalid_temp: 2
missing_key: 2
+---------+-------------+-----------+------+----+----+------------------------------------------------------------------------------------------------------------------------------------+-----------------------+-------------------+----------+
|device_id|temperature_c|vibration_g|status|site|line|raw_json                                                                                                                            |ingest_ts              |event_ts           |date      |
+---------+-------------+-----------+------+----+----+------------------------------------------------------------------------------------------------------------------------------------+-----------------------+-------------------+----------+
|MTR-002  |999.0        |0.3        |OK    |fab1|L02 |{"device_id":"MTR-002","ts":"2025-08-24T00:05:00Z","temperature_c":999.0,"vibration_g":0.3,"status":"OK","site":"fab1","line":"L02"}|2025-08-24 08:32:55.411|2025-08-24 00:05:00|2025-08-2